### удалить какие-то фичи

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from kernels import Kernel
import svm
import time
from sklearn import svm as sksvm

In [2]:
def accuracy(x, y):
    count = 0
    for i in range(len(x)):
        if x[i] == y[i]:
            count += 1
    return count/len(x)

In [3]:
train = pd.read_csv('/Users/grigoriipogorelov/Desktop/train.csv')
train['Date'] = pd.to_datetime(train['Dates'], errors='coerce')
train['Year'] = train['Date'].dt.year
train['Month'] = train['Date'].dt.month
train['WeekOfYear'] = train['Date'].dt.weekofyear
train['Day'] = train['Date'].dt.day
train['Time'] = train['Date'].dt.time
train['Hour'] = train['Date'].dt.hour
# train = train.drop(['Dates','Date','Time','Resolution','Descript','Address'], axis=1)
train = train.drop(['Dates','Date','Time','Resolution','Descript','Address','DayOfWeek', 'WeekOfYear'], axis=1)

# cat_cols = ['DayOfWeek', 'PdDistrict', 'Year', 'Month', 'WeekOfYear', 'Day', 'Hour']
cat_cols = ['PdDistrict', 'Year', 'Month', 'Day', 'Hour']
for cat in cat_cols:
    train = pd.get_dummies(train, columns=[cat])
train.shape

(878049, 93)

In [4]:
train_proportion = 0.02
train_len = len(train)
train = train.sample(frac=1)
train = train.iloc[:int(len(train)*train_proportion)]


cats = ['LARCENY/THEFT', 'OTHER OFFENSES']      ##.   численно вывести
train_0 = train.loc[train['Category'] == cats[0]]
train_1 = train.loc[train['Category'] == cats[1]]
frames = [train_0, train_1]
data = pd.concat(frames)


data = data.as_matrix()
data[data == cats[0]] = 1
data[data == cats[1]] = -1
data = np.array(data, dtype='float')
X_train, X_test, y_train, y_test = train_test_split(data[:, 1:], data[:, 0])

In [5]:
print(X_train.shape)
print(X_test.shape)

(7839, 92)
(2614, 92)


In [6]:
start = time.time()
C = 0.1
trainer = svm.SVMTrainer(kernel=Kernel.linear(), c=C)
model = trainer.train(X_train, y_train)
end = time.time()
print(int((end - start) / 60), ' minutes for training')


start = time.time()
predictions = []
for x in X_test:
    predictions.append(model.predict(x))
end = time.time()
print(int((end - start) / 60), ' minutes for predicting')

     pcost       dcost       gap    pres   dres
 0: -3.7885e+03 -1.9115e+03  6e+04  3e+01  2e-09
 1: -5.8350e+02 -1.8348e+03  3e+03  7e-01  1e-09
 2: -5.1197e+02 -8.8073e+02  4e+02  1e-14  3e-10
 3: -5.5721e+02 -6.8939e+02  1e+02  2e-14  2e-10
 4: -5.6942e+02 -6.1697e+02  5e+01  1e-14  3e-10
 5: -5.7430e+02 -5.9052e+02  2e+01  1e-14  3e-10
 6: -5.7679e+02 -5.7920e+02  2e+00  1e-14  3e-10
 7: -5.7714e+02 -5.7762e+02  5e-01  3e-14  3e-10
 8: -5.7719e+02 -5.7744e+02  3e-01  1e-15  3e-10
 9: -5.7721e+02 -5.7739e+02  2e-01  1e-14  2e-10
10: -5.7722e+02 -5.7734e+02  1e-01  4e-15  2e-10
11: -5.7723e+02 -5.7732e+02  8e-02  1e-14  2e-10
12: -5.7724e+02 -5.7730e+02  6e-02  7e-15  2e-10
13: -5.7725e+02 -5.7728e+02  3e-02  1e-15  2e-10
14: -5.7725e+02 -5.7728e+02  2e-02  3e-14  2e-10
15: -5.7726e+02 -5.7727e+02  2e-02  1e-14  2e-10
16: -5.7726e+02 -5.7727e+02  9e-03  9e-15  2e-10
17: -5.7726e+02 -5.7726e+02  5e-03  2e-14  2e-10
18: -5.7726e+02 -5.7726e+02  2e-03  2e-14  3e-10
19: -5.7726e+02 -5.77

In [7]:
clf = sksvm.SVC()
clf.fit(X_train, y_train)
pred_real_svm = clf.predict(X_test)

In [8]:
print('my accuracy is ', accuracy(y_test, predictions))
print('scikit accuracy is ', accuracy(y_test, pred_real_svm))

my accuracy is  0.6231828615149196
scikit accuracy is  0.6231828615149196


data = 0.02, 0.025
accuracy - идеально совпадает, блять